In [ ]:
# CNN from scratch

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import time

In [ ]:
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

In [ ]:
base_dir = os.path.dirname("./data/affectnethq/")
test_dir = os.path.dirname("./data/test2/")
!find $base_dir -type d -print
!find $test_dir -type d -print

In [ ]:
# AffectNet-HQ dataset
train_dir = base_dir

train_anger_dir = os.path.join(train_dir, 'anger')
train_contempt_dir = os.path.join(train_dir, 'contempt')
train_disgust_dir = os.path.join(train_dir, 'disgust')
train_fear_dir = os.path.join(train_dir, 'fear')
train_happy_dir = os.path.join(train_dir, 'happy')
train_neutral_dir = os.path.join(train_dir, 'neutral')
train_sad_dir = os.path.join(train_dir, 'sad')
train_surprise_dir = os.path.join(train_dir, 'surprise')

In [ ]:
# AffectNet-HQ dataset
num_anger_tr = len(os.listdir(train_anger_dir))
num_contempt_tr = len(os.listdir(train_contempt_dir))
num_disgust_tr = len(os.listdir(train_disgust_dir))
num_fear_tr = len(os.listdir(train_fear_dir))
num_happy_tr = len(os.listdir(train_happy_dir))
num_neutral_tr = len(os.listdir(train_neutral_dir))
num_sad_tr = len(os.listdir(train_sad_dir))
num_surprise_tr = len(os.listdir(train_surprise_dir))

total_train = num_anger_tr+num_disgust_tr+num_contempt_tr+num_fear_tr+num_happy_tr+num_neutral_tr+num_sad_tr+num_surprise_tr

In [ ]:
# AffectNet-HQ dataset
print('total training anger images:', num_anger_tr)
print('total training contempt images:', num_contempt_tr)
print('total training disgust images:', num_disgust_tr)
print('total training fear images:', num_fear_tr)
print('total training happy images:', num_happy_tr)
print('total training neutral images:', num_neutral_tr)
print('total training sad images:', num_sad_tr)
print('total training surprise images:', num_surprise_tr)
print("--")
print("Total training images:", total_train)

In [ ]:
BATCH_SIZE = 100
IMAGE_RES = 150

In [ ]:
# AffectNet-HQ dataset
image_gen_train = ImageDataGenerator(
      rescale=1./255,
      rotation_range=25,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest',
      validation_split=0.2)

train_data_gen = image_gen_train.flow_from_directory(batch_size=BATCH_SIZE,
                                                     directory=train_dir,
                                                     shuffle=True,
                                                     target_size=(IMAGE_RES,IMAGE_RES),
                                                     color_mode="rgb",
                                                     class_mode='categorical',
                                                     subset='training')

val_data_gen = image_gen_train.flow_from_directory(batch_size=BATCH_SIZE,
                                                     directory=train_dir,
                                                     shuffle=True,
                                                     target_size=(IMAGE_RES,IMAGE_RES),
                                                     color_mode="rgb",
                                                     class_mode='categorical',
                                                     subset='validation')

image_gen_test = ImageDataGenerator(rescale=1./255)
test_data_gen = image_gen_test.flow_from_directory(batch_size=BATCH_SIZE,
                                                     directory=test_dir,
                                                     #shuffle=True,
                                                     target_size=(IMAGE_RES,IMAGE_RES),
                                                     color_mode="rgb",
                                                     class_mode='categorical',)

In [ ]:
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip(images_arr, axes):
        ax.imshow(img)
    plt.tight_layout()
    plt.show()

In [ ]:
augmented_images = [train_data_gen[0][0][0] for i in range(5)]
plotImages(augmented_images)

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(2)
])

In [ ]:
#model = tf.keras.models.Sequential([
#    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(IMAGE_RES, IMAGE_RES, 3)),
#    tf.keras.layers.BatchNormalization(),
#    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
#    tf.keras.layers.BatchNormalization(),
#    tf.keras.layers.Conv2D(32, (5,5), strides=2, activation='relu'),
#    tf.keras.layers.Dropout(0.4),
#    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
#    tf.keras.layers.BatchNormalization(),
#    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
#    tf.keras.layers.BatchNormalization(),
#    tf.keras.layers.Conv2D(64, (5,5), strides=2, activation='relu'),
#    tf.keras.layers.Dropout(0.4),
#    tf.keras.layers.Flatten(),
#    tf.keras.layers.Dense(512, activation='relu'),
#    tf.keras.layers.BatchNormalization(),
#    tf.keras.layers.Dropout(0.4),
#    tf.keras.layers.Dense(8, activation="softmax")
#])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),#from_logits=True),
              metrics=['accuracy'])

In [ ]:
start_time = time.time()

my_callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=5),
#    tf.keras.callbacks.ModelCheckpoint(filepath='./models/model.{epoch:02d}-{val_accuracy:.2f}.h5'),
#    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
]
epochs=55
history = model.fit(
    train_data_gen,
    epochs=epochs,
    validation_data=val_data_gen, 
    #callbacks=my_callbacks
)

print("--- %s minutes ---" % ((time.time() - start_time)/60.))

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
t = time.time()

export_path_sm = "./models/{}.h5".format(int(t))

model.save(export_path_sm)

In [ ]:
# model = tf.keras.models.load_model('models/emotions.h5')

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix

Y_pred = model.predict(test_data_gen)
y_pred = np.argmax(Y_pred, axis=1)
labels = ['anger', 'contempt', 'disgust', 'fear', 'happiness', 'neutral', 'sadness', "surprise"]

cm = confusion_matrix(test_data_gen.classes, y_pred)

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
fig, ax = plt.subplots(figsize=(10,10))
disp.plot(cmap=plt.cm.Blues, ax=ax)
plt.show()

In [ ]:
# Transfer Learning

In [ ]:
import tensorflow_hub as hub

In [ ]:
URL = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/2"

feature_extractor = hub.KerasLayer(URL,
                                   input_shape=(IMAGE_RES, IMAGE_RES,3))

In [ ]:
feature_extractor.trainable = True

In [ ]:
model = tf.keras.Sequential([
  feature_extractor,
  model.add(Dense(512, activation='relu'))
  model.add(Dropout(0.2))
  tf.keras.layers.Dense(8, activation="softmax")
])

model.summary()

In [ ]:
start_time = time.time()

model.compile(
  optimizer='adam',
  loss=tf.keras.losses.CategoricalCrossentropy(),#from_logits=True),
  metrics=['accuracy'])

EPOCHS = 10
history = model.fit(train_data_gen,
                    epochs=EPOCHS,
                    validation_data=val_data_gen,)

print("--- %s minutes ---" % ((time.time() - start_time)/60.))

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(EPOCHS)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
emotions  = ['😡 - anger', '🤢 - disgust', '😱 - fear', '😊 - happiness', '😐 - neutral', '😔 - sadness', "😲 - surprise"]

In [ ]:
predicted_batch = model.predict(test_data_gen)
predicted_batch = tf.squeeze(predicted_batch).numpy()
predicted_ids = np.argmax(predicted_batch, axis=-1)
predicted_class_names = emotions[predicted_ids]
predicted_class_names

In [ ]:
plt.figure(figsize=(10,9))
for n in range(30):
  plt.subplot(6,5,n+1)
  plt.subplots_adjust(hspace = 0.3)
  plt.imshow(image_batch[n])
  color = "blue" if predicted_ids[n] == label_batch[n] else "red"
  plt.title(predicted_class_names[n].title(), color=color)
  plt.axis('off')
_ = plt.suptitle("Model predictions (blue: correct, red: incorrect)")

In [ ]:
t = time.time()

export_path_sm = "./models/model{}.h5".format(int(t))
print(export_path_sm)

tf.keras.models.save_model(model, export_path_sm)